# Web scraping

Cuando necesitamos extraer información publicada en internet, lo ideal es consultar una API, porque:

* Las respuestas contienen información estructurada
* En general, el propio servicio nos da documentación sobre cómo hacer peticiones y qué tipo de información podemos solicitar

Pero muchas veces nos encontramos con información en páginas web (en formato [HTML](https://es.wikipedia.org/wiki/HTML)) que nos gustaría obtener, pero sin API disponible.

Estas páginas `HTML` tienen cierta estructura, aunque con ciertos contras:

* Es más compleja, puede tener muchos niveles de anidamiento
* Es inestable. Están diseñadas para que se vean bien desde el explorador, no para guardar una estructura de consulta. De un día para otro, puede verse alterada por la incorporación de nuevos elementos visuales u otros motivos.
* Puede ser modificada por código cliente (javascript) en diferentes momentos: al cargar la página, al interaccionar con algún elemento, ...

### Ejercicio

Desde tu explorador, consulta el código fuente de una página de tu interés. Por ejemplo, para hacerlo en chrome:

* Accede a la página, p.e. [esta](https://es.wikipedia.org/wiki/HTML).
* Haz click derecho y pulsa `View page source`. Otra opción es pulsar `Inspect`, que además abrirá las herramientas de desarrollador de Chrome, muy útiles para navegar por la estructura de la página.

## Scraping de elementos html

La librería que vamos a utilizar es [Beautiful Soup](https://pypi.org/project/beautifulsoup4/). Nos permite buscar elementos y navegar por la estructura del html fácilmente.

Imaginemos que queremos comparar precios de un determinado modelo de motocicleta de segunda mano. P.e. con [esta búsqueda](https://www.milanuncios.com/motos-de-carretera/duke-390.htm) en milanuncios.

Primero, nos descargamos el html con `requests`.

In [14]:
import requests
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen

In [7]:
import urllib.request
from urllib.request import urlopen
import re

htmlfile =request('https://carnejovenmadrid.com/es/ventajas/permanentes', headers={'User-Agent': 'Mozilla/5.0'})
page = urlopen(htmlfile).read()

NameError: name 'request' is not defined

In [ ]:
page

In [8]:
#Esto da la respuesta de impirmir el contenido
page = requests.get('https://carnejovenmadrid.com/es/ventajas/permanentes')
page

<Response [403]>

In [15]:
url = "https://carnejovenmadrid.com/es/ventajas/permanentes"

req = Request(url, headers = {'User-Agent': 'Mozilla/5.0'})

webpage = urlopen(req).read()

In [ ]:
webpage

Podemos ver el contenido examinando la propiedad `content`.

In [18]:
#Este es 
webpage.content

AttributeError: 'bytes' object has no attribute 'content'

Este contenido es solo texto, no tiene estructura. Aún no podemos hacer búsquedas ni navegar por él.

Para hacerlo, creamos una instancia de `Beautiful Soup` y lo parseamos

In [20]:
#Crear algo con estructura que podamos navegar
soup = BeautifulSoup(webpage, 'html.parser')

In [ ]:
soup

Sobre esto, podemos hacer búsquedas con `find` y `find_all` (o `select_one` y `select` si prefieres utilizar [selectores css](https://en.wikipedia.org/wiki/Cascading_Style_Sheets#Selector)). Sobre nuestro ejemplo, vamos a buscar todos los precios. Examinando el código fuente, vemos que son etiquetas `div` con clase `aditem-price`. 

In [22]:
div_info = soup.find_all('div', class_='info')
div_info

[<div class="info">
 <p>A4 Toner</p>
 </div>, <div class="info">
 <p>Lasik Center Clínica Oftalmológica</p>
 </div>, <div class="info">
 <p>Nattivus</p>
 </div>, <div class="info">
 <p>CaixaForum Madrid</p>
 </div>, <div class="info">
 <p>Museo Arqueológico Nacional</p>
 </div>, <div class="info">
 <p>Museo Cerralbo</p>
 </div>, <div class="info">
 <p>Museo de América</p>
 </div>, <div class="info">
 <p>Museo del Romanticismo</p>
 </div>, <div class="info">
 <p>Museo del Traje</p>
 </div>, <div class="info">
 <p>Museo Nacional Centro de Arte Reina Sofía </p>
 </div>, <div class="info">
 <p>Museo Nacional de Antropología</p>
 </div>, <div class="info">
 <p>Museo Nacional de Artes Decorativas</p>
 </div>]

`find_all` devuelve una lista de elementos. Sobre ellos, podemos hacer:

`children` para sacar el listado de todos los hijos.

In [32]:
div_info[0]

<div class="info">
<p>A4 Toner</p>
</div>

In [52]:
#Se hace un cast a list
list(div_info[0].children)

['\n', <p>A4 Toner</p>, '\n']

`get_text()` para sacar el texto de todos los hijos

In [63]:
div_info[4].p.get_text()

'Museo Arqueológico Nacional'

In [56]:
div_pinfo[0].get_text()

'\nA4 Toner\n'

Por tanto, para sacar el listado de todos los precios podemos hacer:

In [78]:
Aliados = [(div_info.p.get_text()) for div_info in div_info]
Aliados

['A4 Toner',
 'Lasik Center Clínica Oftalmológica',
 'Nattivus',
 'CaixaForum Madrid',
 'Museo Arqueológico Nacional',
 'Museo Cerralbo',
 'Museo de América',
 'Museo del Romanticismo',
 'Museo del Traje',
 'Museo Nacional Centro de Arte Reina Sofía ',
 'Museo Nacional de Antropología',
 'Museo Nacional de Artes Decorativas']

In [ ]:
div_post = soup.find_all('ul', id='posts-masonry' )
div_post


In [158]:
ul = div_post
ul.find_all('li')

AttributeError: ResultSet object has no attribute 'find_all'. You're probably treating a list of items like a single item. Did you call find_all() when you meant to call find()?

In [157]:
# Get the URL
ul.find_all('a')[0].attrs

{'href': '/es/ventajas/permanentes/a4-toner,5461ff6826ac5a64eafce4bf5f975aa7',
 'title': 'Promoción: A4 Toner. Ventaja Novedad, Online.'}

In [ ]:
ul = soup.find("h2", text="Tus resultados de búsqueda:").find_next_sibling("ul")
for item in ul.find_all('li', class_=True):
    print(item)

In [171]:
ul.find_all("li")[0]

<li class="post compras">
<a href="/es/ventajas/permanentes/a4-toner,5461ff6826ac5a64eafce4bf5f975aa7" title="Promoción: A4 Toner. Ventaja Novedad, Online.">
<img alt="" height="205" src="/upload/ofertas/ventajas/A4toner.jpg" width="250"/>
<div class="info">
<p>A4 Toner</p>
</div>
<div class="detalle">
<ul>
<li>
<span class="nota-hide">Ventaja</span>
<strong>  Novedad </strong>
<span class="nota-hide">de la Comunidad de Madrid</span>
</li>
<li>
<span class="nota-hide">Ventaja</span>
<strong>  Online </strong>
<span class="nota-hide">de la Comunidad de Madrid</span>
</li>
</ul>
</div>
</a>
</li>

In [173]:
ul = soup.find("h2", text="Tus resultados de búsqueda:").find_next_sibling("ul")

for li in ul.find_all('li'):
        print(li.text)





A4 Toner




Ventaja
  Novedad 
de la Comunidad de Madrid


Ventaja
  Online 
de la Comunidad de Madrid






Ventaja
  Novedad 
de la Comunidad de Madrid


Ventaja
  Online 
de la Comunidad de Madrid





Lasik Center Clínica Oftalmológica




Ventaja
  Novedad 
de la Comunidad de Madrid






Ventaja
  Novedad 
de la Comunidad de Madrid





Nattivus




Ventaja
  Novedad 
de la Comunidad de Madrid






Ventaja
  Novedad 
de la Comunidad de Madrid





CaixaForum Madrid




Ventaja
  Gratis 
de la Comunidad de Madrid


Ventaja
  Top 
de la Comunidad de Madrid






Ventaja
  Gratis 
de la Comunidad de Madrid


Ventaja
  Top 
de la Comunidad de Madrid





Museo Arqueológico Nacional




Ventaja
  Gratis 
de la Comunidad de Madrid


Ventaja
  Top 
de la Comunidad de Madrid






Ventaja
  Gratis 
de la Comunidad de Madrid


Ventaja
  Top 
de la Comunidad de Madrid





Museo Cerralbo




Ventaja
  Gratis 
de la Comunidad de Madrid


Ventaja
  Top 
de la Comunidad de Madrid








In [ ]:
list(div_post[].children)

In [114]:
div_post[0].li.get_text()

'\n\n\n\nA4 Toner\n\n\n\n\nVentaja\n  Novedad \nde la Comunidad de Madrid\n\n\nVentaja\n  Online \nde la Comunidad de Madrid\n\n\n\n\n'

In [ ]:
Aliados = [(div_info.p.get_text()) for div_info in div_info]
Aliados

Tienes más funciones útiles con pequeños ejemplos [aquí](http://akul.me/blog/2016/beautifulsoup-cheatsheet/)

### Ejercicio

Crea un dataframe de pandas en el que cada fila sea un anuncio y tenga como columnas información que consideres relevante: precio, kilómetros, año, cilindrada, texto del anuncio, ...

In [27]:
div_km = soup.find_all('div', class_='kms')
div_km

[<div class="kms tag-mobile">1.200 kms</div>,
 <div class="kms tag-mobile">13.000 kms</div>,
 <div class="kms tag-mobile">1 kms</div>,
 <div class="kms tag-mobile">12.726 kms</div>,
 <div class="kms tag-mobile">9.700 kms</div>,
 <div class="kms tag-mobile">11.000 kms</div>,
 <div class="kms tag-mobile">1 kms</div>,
 <div class="kms tag-mobile">860 kms</div>,
 <div class="kms tag-mobile">6.000 kms</div>,
 <div class="kms tag-mobile">830 kms</div>,
 <div class="kms tag-mobile">23.000 kms</div>,
 <div class="kms tag-mobile">15.000 kms</div>,
 <div class="kms tag-mobile">7.500 kms</div>,
 <div class="kms tag-mobile">2.760 kms</div>,
 <div class="kms tag-mobile">18.000 kms</div>,
 <div class="kms tag-mobile">38.000 kms</div>,
 <div class="kms tag-mobile">68.000 kms</div>,
 <div class="kms tag-mobile">26.000 kms</div>,
 <div class="kms tag-mobile">10.500 kms</div>]

Para resolver el problema de que unos tienen y otros no precios y otros elementos, tomamos toda la caja del anuncio

In [ ]:
div_anuncios = soup.find_all('div', class_='aditem')
div_anuncios

In [65]:
#Seleccionamos algun anuncio y buscamos algo dentro de el
#list(div_anuncios[0].find('div', class_ = 'aditem-price'))

#div_anuncios[0].find('div', class_ = 'aditem-price').get_text()


list(div_anuncios[0].find('div', class_ = 'aditem-price').children)[0] #Esto saca los dos hijos y saca el primero

#Falta completar para sacar solo el precio con el signo de euros

'4.399'

In [66]:
#Seleccionamos kms
div_anuncios[0].find('div', class_ = 'kms').get_text()

#list(div_anuncios[0].find('div', class_ = 'kms').children)[0]

'1.200 kms'

In [67]:
#Seleccionamos year
div_anuncios[0].find('div', class_ = 'ano').get_text()

#list(div_anuncios[0].find('div', class_ = 'ano').children)[0]

'año 2017'

In [68]:
#Seleccionamos year
div_anuncios[0].find('div', class_ = 'x4').get_text()

'Motos de carretera en León  (LEON)'

In [69]:
#Seleccionamos year
div_anuncios[0].find('div', class_ = 'cc').get_text()

'390 cc'

In [76]:
def eval_info(elemento):
    if elemento is not None:
        return elemento.get_text()
    return None

def get_info(div_anuncios):
    div_precio = div_anuncios.find('div', class_ = 'aditem-price')
    if div_precio is not None:
        precio = list(div_precio.children)[0]
    else:
        precio = None
    
    kms = eval_info(div_anuncios.find('div', class_ = 'kms'))
    cc = eval_info(div_anuncios.find('div', class_ = 'cc'))
    titulo = eval_info(div_anuncios.find('div', class_ = 'x4'))
    year = eval_info(div_anuncios.find('div', class_ = 'ano'))
    
    propiedades = {
        'precio': precio,
        'kms': kms,
        'cc': cc,
        'year': year,
        'titulo': titulo}
    
    return propiedades

get_info(div_anuncios[0])

dic = [get_info(div_anuncios) for div_anuncios in div_anuncios]

In [79]:
#Crear un DataFrame
import pandas as pd
motos = pd.DataFrame(dic)
motos

,cc,kms,precio,titulo,year
0,390 cc,1.200 kms,4.399,Motos de carretera en León (LEON),año 2017
1,390 cc,13.000 kms,3.300,Motos de carretera en Las Palmas de Gran Canar...,año 2015
2,390 cc,1 kms,5.459,Motos de carretera en Valencia (VALENCIA),año 2018
3,375 cc,12.726 kms,3.999,Motos de carretera en toledo (TOLEDO),año 2016
4,390 cc,9.700 kms,4.200,Motos de carretera en Palma (BALEARES),año 2016
5,948 cc,None,8.599,Motos de carretera en mejorada del campo (MAD...,año 2017
6,803 cc,None,8.490,Motos de carretera en mejorada del campo (MAD...,año 2018
7,312 cc,None,3.995,Motos de carretera en mejorada del campo (MAD...,año 2019
8,649 cc,None,6.999,Motos de carretera en mejorada del campo (MAD...,año 2018
9,689 cc,None,6.699,Motos de carretera en mejorada del campo (MAD...,año 2018


In [105]:
import re
#def limpia_kms(fila):
#    if fila.kms is None:
#        return None
#    return int(''join(re.find_all('[0-9]+')), fila.precio)

def limpia_kms(fila):
    if fila.kms is None:
        return None
    fila.kms = fila.kms.replace(' kms', '')
    fila.kms = fila.kms.replace('.', '')
    
    return int(fila.kms)

motos['kms'] = motos.apply(limpia_kms, axis=1)

In [103]:
def limpia_ano(fila):
    if fila.year is None:
        return None
    fila.kms = fila.kms.replace(' kms', '')
    fila.kms = fila.kms.replace('.', '')
    
    return int(fila.kms)

motos['kms'] = motos.apply(limpia_kms, axis=1)

,cc,kms,precio,titulo,year
0,390 cc,1200.0,4.399,Motos de carretera en León (LEON),año 2017
1,390 cc,13000.0,3.300,Motos de carretera en Las Palmas de Gran Canar...,año 2015
2,390 cc,1.0,5.459,Motos de carretera en Valencia (VALENCIA),año 2018
3,375 cc,12726.0,3.999,Motos de carretera en toledo (TOLEDO),año 2016
4,390 cc,9700.0,4.200,Motos de carretera en Palma (BALEARES),año 2016
5,948 cc,NaN,8.599,Motos de carretera en mejorada del campo (MAD...,año 2017
6,803 cc,NaN,8.490,Motos de carretera en mejorada del campo (MAD...,año 2018
7,312 cc,NaN,3.995,Motos de carretera en mejorada del campo (MAD...,año 2019
8,649 cc,NaN,6.999,Motos de carretera en mejorada del campo (MAD...,año 2018
9,689 cc,NaN,6.699,Motos de carretera en mejorada del campo (MAD...,año 2018


### Ejercicio

Modifica el código anterior para que, además de bajarse la página actual, navegue por el resto de pagínas e incorpore también esos anuncios a tu dataframe.

In [ ]:
sigue_procesando = True
numero_pagina = 1

while sigue_procesando:
    #Peticion a la pagina
    page = requests.get('https://www.milanuncios.com/motos-de-carretera/duke-390.htm', params = {'pagina': numero_pagina})
    
    soup = BeautifulSoup(page.content, 'html.parser')
    div_anuncios = soup.find_all('div', class_='aditem')
    



## Scraping de tablas

A menudo, la información que nos interesa descargar está en tablas y nuestro objetivo es importarlas en tablas de Pandas. Esta conversión suele exigir la manipulación del texto, números y fechas contenidas en la tabla original, lo que nos obligará a repasar cómo realizar esas operaciones y aplicarlas a filas y columnas de las tablas.

La estructura que suelen tener la tablas en `html` es:

```
<table>
    <thead>
        <tr>
            <th>Columna A</th>
            <th>Columna B</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>A1</td>
            <td>B1</td>
        </tr>
        <tr>
            <td>A2</td>
            <td>B2</td>
        </tr>
    </tbody>
</table>   
```

## Scraping de tablas

A menudo, la información que nos interesa descargar está en tablas y nuestro objetivo es importarlas en tablas de Pandas. Esta conversión suele exigir la manipulación del texto, números y fechas contenidas en la tabla original, lo que nos obligará a repasar cómo realizar esas operaciones y aplicarlas a filas y columnas de las tablas.

La estructura que suelen tener la tablas en `html` es:

```
<table>
    <thead>
        <tr>
            <th>Columna A</th>
            <th>Columna B</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>A1</td>
            <td>B1</td>
        </tr>
        <tr>
            <td>A2</td>
            <td>B2</td>
        </tr>
    </tbody>
</table>   
```

Necesitaremos los siguientes módulos además de `requests` y `BeautifulSoup` importados anteriormente:

In [ ]:
import pandas as pd
import re

Primero, hacemos una petición para descargar la página de interés (que contiene las cotizaciones de las acciones del IBEX 35 en tiempo _casi_ real).

In [ ]:
base_url = "https://www.eleconomista.es/indice/IBEX-35"
res = requests.get(base_url)
contenido = res.content
contenido

La siguiente línea procesa el HTML de la página que hemos descargado:

In [155]:
soup = BeautifulSoup(contenido, "html.parser")

Una vez procesado el HTML, es posible buscar elementos dentro de él. En particular, podemos buscar los elementos de tipo `table`, es decir, tablas.

In [168]:
tablas = soup.find_all('table')

El objeto `tablas` contiene todas las tablas presentes en la página. Hay que tener cuidado con dichas tablas porque muchas páginas utilizan elementos de tipo `table` para estructurar el contenido. Por eso, en algunas páginas, aunque parezca haber una única tabla, puede haber otras con una información no interesante que toca descartar.

In [169]:
len(tablas)

3

Podemos extraer las filas de todas estas tablas

In [181]:
lineas[1].find_all('td')[0] #Selecciona el primer td

<td class="footable-first-visible" itemscope="" itemtype="http://schema.org/SiteNavigationElement"><a href="/empresa/ACCIONA" itemprop="url">ACCIONA</a></td>

In [182]:
#Hacemos lo mismo anterior para el resto del script
lineas = [x for tabla in tablas for x in tabla.find_all('tr')]
lineas

[<tr class="footable-header"><th class="footable-first-visible" href="/indice/IBEX-35/resumen/Nombre/descendente">Nombre</th><th>Precio</th><th data-breakpoints="" data-title=""></th><th data-breakpoints=""><a href="/indice/IBEX-35/resumen/Mejores">Var. (%)</a></th><th data-breakpoints="xs">Var. (€)</th><th data-breakpoints="sm xs" data-title="Volumen (€)"><a href="/indice/IBEX-35/resumen/Volumen">Volumen (€)</a></th><th data-breakpoints="sm xs" data-title="Cap."><a href="/indice/IBEX-35/resumen/Capitalizacion">Capitalización</a><sup>(1)</sup></th><th data-breakpoints="sm xs" data-title="PER"><a href="/indice/IBEX-35/resumen/PER">PER</a></th><th data-breakpoints="sm xs" data-title="Rent. /Div."><a href="/indice/IBEX-35/resumen/Rentabilidad-Dividendo">Rent. /Div.</a></th><th class="footable-last-visible" data-breakpoints="sm xs" data-title="Hora"> Hora </th></tr>,
 <tr><td class="footable-first-visible" itemscope="" itemtype="http://schema.org/SiteNavigationElement"><a href="/empresa/AC

para luego extraer los contenidos de cada fila individualmente haciendo

In [183]:
datos = [[x.text for x  in linea.find_all('td')] for linea in lineas]
datos

[[],
 ['ACCIONA',
  '84,52',
  '',
  '-0,38%',
  '-0,32',
  '2.072.710,26',
  '4.836,14',
  '16,62',
  '3,96%',
  '12:20'],
 ['ACERINOX',
  '9,63',
  '',
  '+0,52%',
  '0,05',
  '3.843.344,12',
  '2.652,46',
  '11,56',
  '5,08%',
  '12:20'],
 ['ACS',
  '35,93',
  '',
  '-0,77%',
  '-0,28',
  '2.615.204,38',
  '11.302,75',
  '11,24',
  '4,79%',
  '12:20'],
 ['AENA',
  '150,25',
  '',
  '-1,02%',
  '-1,55',
  '2.006.341,40',
  '22.605,00',
  '17,01',
  '4,64%',
  '12:18'],
 ['AMADEUS',
  '66,50',
  '',
  '+0,61%',
  '0,40',
  '7.323.668,60',
  '29.172,92',
  '22,76',
  '2,03%',
  '12:20'],
 ['ARCELORMITTAL',
  '20,77',
  '',
  '+0,58%',
  '0,12',
  '3.073.045,46',
  '21.168,73',
  '0,00',
  '0,00%',
  '12:19'],
 ['BANKIA',
  '2,52',
  '',
  '+0,88%',
  '0,02',
  '2.767.628,56',
  '7.755,60',
  '10,71',
  '5,24%',
  '12:19'],
 ['BANKINTER',
  '6,67',
  '',
  '+0,54%',
  '0,04',
  '3.269.676,47',
  '6.002,63',
  '10,86',
  '4,60%',
  '12:20'],
 ['BBVA',
  '5,21',
  '',
  '-0,27%',
  '-0,01

Podemos inspeccionar parte del objeto resultante:

In [184]:
datos[0:3]

[[],
 ['ACCIONA',
  '84,52',
  '',
  '-0,38%',
  '-0,32',
  '2.072.710,26',
  '4.836,14',
  '16,62',
  '3,96%',
  '12:20'],
 ['ACERINOX',
  '9,63',
  '',
  '+0,52%',
  '0,05',
  '3.843.344,12',
  '2.652,46',
  '11,56',
  '5,08%',
  '12:20']]

Vemos que hay filas que contienen la información de interés junto con otras que contienen cabeceras y otra información irrelevante. En general, la situación puede ser más complicada y se hace necesario estudiar el objeto `tablas` para seleccionar la de interés.

En nuestro caso, podemos filtrar las líneas menos relevantes así:

In [185]:
datos = [x for x in datos if len(x) > 0]

Finalmente, podemos crear una tabla de Pandas:

In [186]:
datos = pd.DataFrame(datos)
datos

,0,1,2,3,4,5,6,7,8,9
0,ACCIONA,"84,52",,"-0,38%","-0,32","2.072.710,26","4.836,14","16,62","3,96%",12:20
1,ACERINOX,"9,63",,"+0,52%","0,05","3.843.344,12","2.652,46","11,56","5,08%",12:20
2,ACS,"35,93",,"-0,77%","-0,28","2.615.204,38","11.302,75","11,24","4,79%",12:20
3,AENA,"150,25",,"-1,02%","-1,55","2.006.341,40","22.605,00","17,01","4,64%",12:18
4,AMADEUS,"66,50",,"+0,61%","0,40","7.323.668,60","29.172,92","22,76","2,03%",12:20
5,ARCELORMITTAL,"20,77",,"+0,58%","0,12","3.073.045,46","21.168,73","0,00","0,00%",12:19
6,BANKIA,"2,52",,"+0,88%","0,02","2.767.628,56","7.755,60","10,71","5,24%",12:19
7,BANKINTER,"6,67",,"+0,54%","0,04","3.269.676,47","6.002,63","10,86","4,60%",12:20
8,BBVA,"5,21",,"-0,27%","-0,01","26.527.312,00","34.833,04","8,10","5,39%",12:20
9,CAIXABANK,"2,97",,"+1,19%","0,04","23.833.015,33","17.758,89","8,19","6,29%",12:20


#### Ejercicio

Usa los elementos `th` de la primera fila de las tablas para extraer nombres para las columnas de la tabla. 

In [187]:
lineas[0].find_all('th')

[<th class="footable-first-visible" href="/indice/IBEX-35/resumen/Nombre/descendente">Nombre</th>,
 <th>Precio</th>,
 <th data-breakpoints="" data-title=""></th>,
 <th data-breakpoints=""><a href="/indice/IBEX-35/resumen/Mejores">Var. (%)</a></th>,
 <th data-breakpoints="xs">Var. (€)</th>,
 <th data-breakpoints="sm xs" data-title="Volumen (€)"><a href="/indice/IBEX-35/resumen/Volumen">Volumen (€)</a></th>,
 <th data-breakpoints="sm xs" data-title="Cap."><a href="/indice/IBEX-35/resumen/Capitalizacion">Capitalización</a><sup>(1)</sup></th>,
 <th data-breakpoints="sm xs" data-title="PER"><a href="/indice/IBEX-35/resumen/PER">PER</a></th>,
 <th data-breakpoints="sm xs" data-title="Rent. /Div."><a href="/indice/IBEX-35/resumen/Rentabilidad-Dividendo">Rent. /Div.</a></th>,
 <th class="footable-last-visible" data-breakpoints="sm xs" data-title="Hora"> Hora </th>]

In [188]:
head = [[x.text for x  in linea.find_all('th')] for linea in lineas][0]
head

['Nombre',
 'Precio',
 '',
 'Var. (%)',
 'Var. (€)',
 'Volumen (€)',
 'Capitalización(1)',
 'PER',
 'Rent. /Div.',
 ' Hora ']

In [189]:
datos.columns = head
datos

,Nombre,Precio,,Var. (%),Var. (€),Volumen (€),Capitalización(1),PER,Rent. /Div.,Hora
0,ACCIONA,"84,52",,"-0,38%","-0,32","2.072.710,26","4.836,14","16,62","3,96%",12:20
1,ACERINOX,"9,63",,"+0,52%","0,05","3.843.344,12","2.652,46","11,56","5,08%",12:20
2,ACS,"35,93",,"-0,77%","-0,28","2.615.204,38","11.302,75","11,24","4,79%",12:20
3,AENA,"150,25",,"-1,02%","-1,55","2.006.341,40","22.605,00","17,01","4,64%",12:18
4,AMADEUS,"66,50",,"+0,61%","0,40","7.323.668,60","29.172,92","22,76","2,03%",12:20
5,ARCELORMITTAL,"20,77",,"+0,58%","0,12","3.073.045,46","21.168,73","0,00","0,00%",12:19
6,BANKIA,"2,52",,"+0,88%","0,02","2.767.628,56","7.755,60","10,71","5,24%",12:19
7,BANKINTER,"6,67",,"+0,54%","0,04","3.269.676,47","6.002,63","10,86","4,60%",12:20
8,BBVA,"5,21",,"-0,27%","-0,01","26.527.312,00","34.833,04","8,10","5,39%",12:20
9,CAIXABANK,"2,97",,"+1,19%","0,04","23.833.015,33","17.758,89","8,19","6,29%",12:20


In [ ]:
#Opcion 2
names = soup.fins_all('tr', class_ = 'footable-header')
names = [[x.text for x  in name.find_all('th')] for name in names][0]

#### Ejercicio

Elimina las columnas irrelevantes y cambia los nombres de las columnas por otros breves y sin caracteres extraños o que dificulten el posproceso.

In [190]:
#Eliminar la columna vacia
del datos['']

datos

,Nombre,Precio,Var. (%),Var. (€),Volumen (€),Capitalización(1),PER,Rent. /Div.,Hora
0,ACCIONA,"84,52","-0,38%","-0,32","2.072.710,26","4.836,14","16,62","3,96%",12:20
1,ACERINOX,"9,63","+0,52%","0,05","3.843.344,12","2.652,46","11,56","5,08%",12:20
2,ACS,"35,93","-0,77%","-0,28","2.615.204,38","11.302,75","11,24","4,79%",12:20
3,AENA,"150,25","-1,02%","-1,55","2.006.341,40","22.605,00","17,01","4,64%",12:18
4,AMADEUS,"66,50","+0,61%","0,40","7.323.668,60","29.172,92","22,76","2,03%",12:20
5,ARCELORMITTAL,"20,77","+0,58%","0,12","3.073.045,46","21.168,73","0,00","0,00%",12:19
6,BANKIA,"2,52","+0,88%","0,02","2.767.628,56","7.755,60","10,71","5,24%",12:19
7,BANKINTER,"6,67","+0,54%","0,04","3.269.676,47","6.002,63","10,86","4,60%",12:20
8,BBVA,"5,21","-0,27%","-0,01","26.527.312,00","34.833,04","8,10","5,39%",12:20
9,CAIXABANK,"2,97","+1,19%","0,04","23.833.015,33","17.758,89","8,19","6,29%",12:20


In [ ]:
datos.columns = ['']

In [210]:
datos.columns = ['nombre', 'precio', 'var_pct', 'var_eur', 'volumen', 'capitalizacion', 'per', 'rent_div', 'hora']
datos.head()

,nombre,precio,var_pct,var_eur,volumen,capitalizacion,per,rent_div,hora
0,ACCIONA,84.52,"-0,38%","-0,32","2.072.710,26","4.836,14","16,62","3,96%",12:20
1,ACERINOX,9.63,"+0,52%","0,05","3.843.344,12","2.652,46","11,56","5,08%",12:20
2,ACS,35.93,"-0,77%","-0,28","2.615.204,38","11.302,75","11,24","4,79%",12:20
3,AENA,150.25,"-1,02%","-1,55","2.006.341,40","22.605,00","17,01","4,64%",12:18
4,AMADEUS,66.50,"+0,61%","0,40","7.323.668,60","29.172,92","22,76","2,03%",12:20


#### Ejercicio

Cambia el formato de las columnas adecuadamente: convierte a numéricas, etc., las columnas que lo requieran.

In [1]:
Datos = datos.copy

#type(datos['Precio'])
#type(datos.Precio)
#type(datos['Hora'])

#Eliminar puntos y comas
#numero = '2,34'
def limpia_num(numero):
    #eliminar puntos porcentajes y signos mas
    numero = numero .replace('.', '')
    numero = numero .replace('%', '')
    numero = numero .replace('+', '')

    #reemplzar comas por puntos
    numero = numero .replace(',', '.')

    #casting a float
    return float(numero)

# datos['precio'] = datos.apply(lambda fila: limpia_numero(fila.precio), axis=1)
columnas_a_limpiar = ['precio', 'var_pct', 'var_eur', 'volumen', 'capitalizacion', 'per', 'rent_div']
for col in columnas_a_limpiar:
    datos[col] = datos.apply(lambda fila: limpia_num(fila[col]), axis=1)
datos

NameError: name 'datos' is not defined

## Riesgos del scraping

El scraping es una técnica potente pero tiene varios contras:

* Implica mayor tiempo de desarrollo y mayor esfuerzo en la limpieza de datos (en comparación con otras fuentes como APIs, BDs, ...)
* Si hay que scrapear gran cantidad de páginas, es lento
* Los servidores objetivo de nuestro scraping pueden tener técnicas para evitarlo. Por ejemplo, bloquear la IP temporalmente o introducir delays en las respuestas si hacemos muchas peticiones en poco tiempo. Esto pasa especialmente en las grandes webs recelosas de sus datos (p.e. linkedin, amazon, ...).
* El código de scraping escrito hoy puede no funcionar mañana, si la web destino cambia nombres, etiquetas o estructura. Si se sube a producción para lanzarlo periódicamente, hay que ser conscientes de que en algún momento fallará, y establecer mecanismos de alerta

## Javascript

Es posible que te encuentres con algún caso en el que no puedas descargar tal cual el html y parsearlo, principalmente por dos motivos:

* La estructura de la página se genera parcial o totalmente en cliente
* Debemos interactuar con algún elemento para mostrar la información que queremos (p.e. completar un campo de búsuqeda, hacer click en algún botón, ...)

En estos casos, hay que ejecutar en un navegador local el código javascript de la página destino. Para esta tarea, puedes utilizar [Selenium]().

[Aquí](https://medium.freecodecamp.org/better-web-scraping-in-python-with-selenium-beautiful-soup-and-pandas-d6390592e251) un post con un ejemplo de uso.